In [1]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt

# Методы Монте-Карло

В лабораторной [про квадратурные формулы](Integration.ipynb) мы пытались посчитать объем четырехмерного шара
методами трапеций и Симпсона. Абсолютная погрешность этих метод имеет асимптотику $h^d$ при шаге решетки $h\to0$, 
где $d$ - порядок метода.
Число точек, в которых должно быть вычислено подинтегральное выражение, изменяется как $h^{-D}$, где $D=4$ - размерность пространства. 
Для больших размерностей $D$ это делает использование квадратурных формул почти невозможным.
Существует однако возможность использовать небольшое число случайно выбранных точек для оценки интеграла, вмество построения мелкой решетки.
Этот подход базируется на связи интегралов и средних значений случайных величин:
$$
E_p[f]=\int_{\Omega} f(x)p(x)dx,
$$
где $f$ случайная величина, заданная на множестве $\Omega$ элементарных событий с плотностью распределения $p$,
$E_p[f]$ обозначает среднее значение $f$ на этом распределении.
Если мы имеем возможность генерировать выборку значений $x_k$ случайной величины $X$, отвечающей $p$, 
то среднее значение оценивается статистикой:
$$
E_p[f] \approx M_N = \frac{1}{N}\sum_{n=1}^N f(x_n),
$$
что дает нам связь между интегралом и суммой по случайно выбранным точкам.
Методы интегрирования на основе случайных выборок обычно называют методами Монте-Карло. 

Среднее арифметическое $M_N$ также является случайной величиной,
дисперсия которой оценивается как 
$$
\sigma(M_n)^2 = \frac{\sigma_p(f)^2}{N},
$$
где $\sigma_p(f)^2$ обозначает дисперсию случайной величины $f$.
Согласно этой формуле ошибка оценивания интеграла средним арфиметическим убывает как $N^{-\frac12}$,
что довольно медленно, однако эта скорость не зависит от размерности пространства, 
что делает метод Монте-Карло предпочтительным для задач большой размерности.

Для вычисления обьема четырехмерного шара $B=\{x\in\mathbb R^4\colon |x|^2\leq 1\},$
мы возьмем случайную величину $X$ равномерно распределенную на кубе $\Omega=[-1,1]^{\times 4}$.
Воспользуемся очевидными тождествами:
$$\frac{V(B)}{V(\Omega)}=P\{X\in B\}=E[1_B],$$
где $1_B$ - характеристическая функция множества $B$:
$$
1_B(x)=\begin{cases}
1,&x\in B,\\
0,&x\notin B.
\end{cases}
$$

In [200]:
# Для семплирования разных распределений удобно использовать пакет numpy.random.
# Создадим выборку из N=10000 равномерно распределенных на Omega значений.
N = 100000 # Размер выборки
D = 6 # Размерность пространства
x = np.random.rand(N, D)*2-1
# Найдем значения, попадающие в шар B
is_x_in_B = np.sum(x**2, axis=1)<1
# Посчитаем число попавших в B значений
x_in_B_count = np.sum(is_x_in_B)
# Оцениваем вероятность попасти в шар
p_x_in_B = x_in_B_count / N
# Обьем куба 
V_Omega = 2 ** D
# Наконец искомый интеграл равен
V_B_estimated = p_x_in_B * V_Omega
# Аналитическое значение интеграла
V_B_analytic = np.power(np.pi, D/2) / scipy.special.gamma(D/2+1)
print("Exact volume", V_B_analytic)
print("Estimated volume", V_B_estimated)
print("Relative error", (V_B_estimated-V_B_analytic)/V_B_analytic)


Exact volume 5.167712780049969
Estimated volume 5.18144
Relative error 0.0026563434413431476


## Задания:

1. Вычислите методом Монте-Карло объемы $D$-мерных шаров радиуса $R=1$ для $D=1\ldots 15$ с выборками размера $N=1\ldots 10^6$.
Постройте график зависимости средней квадратической относительной ошибки объема от $N$ для разных $D$. Оцените асимптотику ошибки при $N\to\infty$. 

1. Теоретически обоснуйте наблюдаемую зависимость средней квадратической ошибки от размера выборки. 

1. Как можно оценить предельную ошибку вычисления интеграла методом Монте-Карло? 

1. Сравните ошибку метода Монте-Карло с результатами лабораторной [про квадратурные формулы](Integration.ipynb). Для каких размерностей $D$ метод Монте-Карло оказывается точнее, чем вычисление через произведение составных формул Симпсона по каждой из координат?

# Уменьшение ошибки метода Монте-Карло

Предположим мы хотим вычислить интеграл

$$I=\int_{\Omega} f(x)dx,$$

методом Монте-Карло, для этого мы представляем его в виде среднего математического величины $f(x)/p(x)$,
на плотности распределения $p$:

$$
I = E_p[f/p] = \int_{\Omega}\frac{f(x)}{p(x)}p(x)dx.
$$

Пусть $x_n$ выборка случайной величины $X$ с плотностью $p$, тогда 
$$
I\approx M_N=\frac{1}{N}\sum_{n=1}^N \frac{f(x_n)}{p(x_n)}.
$$
Дисперсия $M_N$ выражает меру ошибки:
$$
\sigma(M_N)^2 = \frac{1}{N}\sigma_p(f/p)^2=\frac{1}{N}\int_{\Omega} \left(\frac{f(x)}{p(x)}-I\right)^2 p(x)dx.
$$
Можно видеть, что хотя для всех $p$ среднее значение $M_N$ совпадает с искомым интегралом $I$,
величина ошибка существенно зависит от выбора $p$.
Идельным является выбор $p$ пропорциональным функии $f$, но генерация произвольного распределения само по себе является сложной задачей.
Поэтому на практике желательно выбирать известное распределение, имеющую плотность как можно более близко повторяющую $f$.
Данный метод является вариацией [выборки по значимости](http://ru.wikipedia.org/wiki/Выборка_по_значимости).

Попробуем улучшить точность вычислений объема шара, отталкиваясь от более естественного распределения, чем равномерное на кубе.
Наш предыдущий подход давал точный ответ для $D=1$. 
Теперь мы будем опираться на известное нам значение площади круга, и получим метод, дающий точный ответ для $D=2$.
Заметим, что если мы имеем пару случайных величин $\mu$, $\nu$, равномерно распределенных на интервале $[0,1]$,
то мы можем получить случайную точку $(x,y)$, равномерно распределенную на круге $x^2+y^2\leq 1$:
$$
\begin{cases}
x=r\cos\phi,\\
y=r\sin\phi,\\
\end{cases}
\begin{cases}
\phi=2\pi\mu,\\
r=\sqrt{\nu}.\\
\end{cases}
$$
Плотность распределения для $(x,y)$ равна 
$$
p(x,y)=\frac{1}{\pi}\theta(x^2+y^2),\quad
\theta(a)=\begin{cases}1,& a<1\\0,&\text{в противном случае}\end{cases}
$$
Идеальным распределением для вычисления обьема многомерного шара было бы равномерное распределение на этом шаре.
В качестве приближения этого распределения мы берем произведение равномерных распределений на кругах:
$$
p(x)=p(x_1,x_2)p(x_3,x_4)\cdots
$$

In [254]:
# Сгенерируем приближенное распределение
DHalf = int(D/2)
phi = 2*np.pi*np.random.rand(N, DHalf)
r = np.sqrt(np.random.rand(N, DHalf))
x = np.empty((N,D))
x[:,:2*DHalf:2]  = r*np.cos(phi)
x[:,1:2*DHalf:2] = r*np.sin(phi)
if D%2==1: x[:,-1] = np.random.rand(N)*2-1
# Вычисляем плотность распределения
p = np.power(np.pi, -DHalf)
if D%2==1: p /= 2
# Вычисляем величину f/p
f = np.array(np.sum(x**2, axis=1)<1, dtype=np.float)
f_over_p = f / p
# Вычисляем интеграл
V_B_second = np.mean(f_over_p)
# Вычисляем ошибку
V_B_analytic = np.power(np.pi, D/2) / scipy.special.gamma(D/2+1)
print("Exact volume", V_B_analytic)
print("Estimated volume", V_B_second)
print("Relative error", (V_B_second-V_B_analytic)/V_B_analytic)


Exact volume 5.167712780049969
Estimated volume 5.124097284186347
Relative error -0.008440000000000074


## Задания:

1. Постройте график средней ошибки от размера выборки. Сравните с ошибкой прошлой попытки.

1. Теоретически оцените величину уменьшения ошибки при переходе ко второму методу.

1. Предложите свой способ уменьшения вариации оценки обьема шара по методу Монте-Карло.

## Замечание:

В методы выше тяжело было найти плотность распределения $p$, сгенерировать выборку величины с данным распределением часто проще.
Например, равномерное распределение на шаре $B$ можно получить из выборки $X_k$ равномерно распределенной на кубе $\Omega$ случайной величины,
отбрасывая все точки $X_k$, не лежащие в $B$, что является частным случаем метода [алгоритма_Метрополиса-Гастингса](http://ru.wikipedia.org/wiki/Алгоритм_Метрополиса_—_Гастингса). Оказывается в ряде случаев достаточно уметь строить выборку для решения содержательных задач, как показано в следующем разделе.

# Метод 

## Задания:

1. Убедитесь, что ваш код хорошо структуирован, в частности: (a) одни и те же действия не выполняются в разных частях кода; (б) все вычисления разбиты на функции, каждая из которых имеет четкое и понятное предназначение и легко читается; (с) все параметры указаны явно, каждый параметр задается ровно в одном месте.